# Sizing for MOSFET Diode Example

**Copyright 2024 Harald Pretl**

Licensed under the Apache License, Version 2.0 (the “License”); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

*Important note: Numpy version 1.x.x is required; an error is thrown for Numpy 2.x.x*

In [1]:
# Read table data
from pygmid import Lookup as lk
import numpy as np
lv_nmos = lk('sg13_lv_nmos.mat')
lv_pmos = lk('sg13_lv_pmos.mat')
# List of parameters: VGS, VDS, VSB, L, W, NFING, ID, VT, GM, GMB, GDS, CGG, CGB, CGD, CGS, CDD, CSS, STH, SFL
# If not specified, minimum L, VDS=max(vgs)/2=0.9 and VSB=0 are used 

In [2]:
# Define the given values
id_spec = 20e-6
gm_id_spec = 10
L_spec = 0.13

In [3]:
# We can calculate the gm directly
gm = gm_id_spec * id_spec
print('gm =', gm/1e-3, 'mS')

gm = 0.2 mS

In [4]:
vgs = lv_nmos.look_upVGS(GM_ID=gm_id_spec, L=L_spec, VDS=0.75, VSB=0.0)
vgs = lv_nmos.look_upVGS(GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0.0)

In [5]:
# The gm_gds we look up and calculate gds from it
gm_gds = lv_nmos.lookup('GM_GDS', GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0)
gds = gm / gm_gds
print('gds =', gds/1e-6, 'uS')

gds = 10.87927113537607 uS

In [6]:
# Find f_T (which is not stored directly, but we can find the gm to gate capacitance ratio)
gm_cgg = lv_nmos.lookup('GM_CGG', GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0)
f_T = gm_cgg / (2*np.pi)
print('Cgg =', gm/gm_cgg/1e-15, 'fF')
print('f_T =', f_T/1e9, 'GHz')


Cgg = 1.3850378609536809 fF
f_T = 22.982035015607114 GHz

In [7]:
# Find the W of the diode transistor
id_w = lv_nmos.lookup('ID_W', GM_ID=gm_id_spec, L=L_spec, VDS=vgs, VSB=0)
w = id_spec / id_w
print('W =', w, 'um, rounded W =', round(w*2)/2, 'um')

W = 0.8609408128056297 um, rounded W = 1.0 um

In [8]:
# Let's now find the other interesting values
# Lookup normalized noise power spectral densisties and denormalize with the gm from above
sth = lv_nmos.lookup('STH_GM', VGS=vgs, L=L_spec, VDS=vgs, VSB=0)*gm
sfl = lv_nmos.lookup('SFL_GM', VGS=vgs, L=L_spec, VDS=vgs, VSB=0)*gm

gamma = sth/(4*1.38e-23*300*gm)

f_co = sfl/sth
print('V_GS =', vgs, 'V')
print('gamma =', gamma)
print('f_co =', f_co/1e6, 'MHz')

V_GS = 0.5907233171282525 V
gamma = 0.809081311364605
f_co = 15.842469386921188 MHz